# Multi-Agent Resume Tailoring System
### Capstone Project - Google & Kaggle AI Agent Intensive Course

## Project Overview

This project implements a multi-agent system using Google's Agent Development Kit (ADK). The pipeline automates resume tailoring by combining:

1. **Resume Intake Agent** – extracts and standardizes resume content  
2. **Job Research Agent** – performs live search to identify real job requirements  
3. **Resume Rewrite Agent** – merges resume + job insights into a tailored final resume  
4. **Polishing Agent** – refines clarity, style, and conciseness

## Architecture Diagram

## Agent Definitions

## Pipeline Design

## 

In [2]:
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"API Authentication Error, please make sure you have setup your .env with the correct GOOGLE_API_KEY: {e}")

✅ Gemini API key setup complete.


In [29]:
from google.adk.agents import Agent, LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.adk.models import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
import google.generativeai as genai

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

🧹 Cleaned up logger.log
✅ Logging configured


In [9]:
# Automatic retries to contact the Gemini API if it fails
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agent Instructions

Each agent recieves a different set of instructions tailored to its role. This seperation is key for improving perforance, security, and manageability as we learned in the course.

In [51]:
INTAKE_INSTRUCTION = """
    Extract structured resume info (education, skills, projects, experience).
    Return bullet points without rewriting or inventing details.
"""

JOB_RESEARCH_INSTRUCTION = """
    Perform a Google search for the given job title/company.
    Summarize required skills, preferred qualifications, and responsibilities.
"""

REWRITE_INSTRUCTION = """
    Combine the parsed resume + job research.
    Rewrite the resume using strong, tailored bullet points.
    Do NOT invent false experience.
"""

POLISH_INSTRUCTION = """
    Clean, format, and simplify the improved resume.
    Ensure conciseness, clarity, and consistent style.
    Make sure to capture the justification/explanation section.
"""

print("✅ Agents instructions created")

✅ Agents instructions created


## Defining Agents and Runners

In [46]:
resume_intake_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeIntakeAgent",
    instruction=INTAKE_INSTRUCTION,
    tools=[]
)

job_research_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="JobResearchAgent",
    instruction=JOB_RESEARCH_INSTRUCTION,
    tools=[google_search]
)

resume_rewrite_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeRewriteAgent",
    instruction=REWRITE_INSTRUCTION,
    tools=[]
)

polish_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="PolishAgent",
    instruction=POLISH_INSTRUCTION,
    tools=[]
)

# Defining Runners
intake_runner = InMemoryRunner(resume_intake_agent)
job_runner = InMemoryRunner(job_research_agent)
rewrite_runner = InMemoryRunner(resume_rewrite_agent)
polish_runner = InMemoryRunner(polish_agent)

print("✅ Agents + Runners created")

✅ Agents + Runners created


## Multi-Agent Pipeline
### Helper function for supressing output (optional) + Agent pipeline

In [49]:
import contextlib
import io

async def silent_run_debug(runner, text):
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        events = await runner.run_debug(text)

    last_text = None

    for event in events:
        if hasattr(event, "content") and event.content:
            parts = event.content.parts
            if parts and hasattr(parts[0], "text"):
                last_text = parts[0].text

    return last_text

async def agent_pipeline(resume_text, job_query):
    intake_output = await silent_run_debug(intake_runner, resume_text)
    job_output = await silent_run_debug(job_runner, job_query)

    resume_prompt = f"""
    Parsed resume:
    {intake_output}

    Job research:
    {job_output}

    Rewrite the resume using BOTH.

    Additionally, include a section beneath that explains YOUR decision making.
    """
    rewrite_output = await silent_run_debug(rewrite_runner, resume_prompt)
    polish_output = await silent_run_debug(polish_runner, rewrite_output)

    return polish_output

print("✅ Pipeline + helper functions created")

✅ Pipeline + helper functions created


### Test Example + Demo

In [50]:
from pathlib import Path
from IPython.display import Markdown, display

# Loading in the demo resume
resume_path = Path("data/alex_carter_demo_resume.txt")

with open(resume_path, "r") as file:
    resume_text = file.read()

job_query = input("Enter a job (e.g., 'Tesla SWE Intern 2026'): ")

result = await agent_pipeline(resume_text, job_query)

display(Markdown(f"## Polished Resume\n\n{result}"))

Enter a job (e.g., 'Tesla SWE Intern 2026'):  Tesla Data Science Intern 2026


## Polished Resume

**[Your Name]**
[Your Phone Number] | [Your Email Address] | [Your LinkedIn Profile URL (Optional)] | [Your GitHub Profile URL (Optional)] | Fremont, CA

---

**Education**

**University of California, San Diego** — B.S. in Computer Science, Expected June 2026
*   **Relevant Coursework:** Data Structures & Algorithms, Machine Learning, Operating Systems, Database Systems, Software Engineering

---

**Technical Skills**

*   **Languages:** Python, Java, C++, SQL, JavaScript
*   **Machine Learning:** PyTorch, TensorFlow, LSTM Networks (Familiarity with concepts: Transformers, CNNs, GANs, Sequence Models)
*   **Backend & APIs:** Node.js, Flask, RESTful APIs, Distributed Systems, Docker
*   **Frontend:** React
*   **Data Pipelines & Tools:** Git, GitHub Actions, MongoDB (Familiarity with data wrangling and workflow management concepts)
*   **Data Visualization:** Conceptual understanding; eager to apply through tools like Tableau
*   **Statistical Concepts:** Understanding applicable to data analysis and model development

---

**Experience**

**Software Engineering Intern — Horizon Analytics** | June 2024 – September 2024
*   Developed and implemented critical data processing modules for a high-throughput telemetry monitoring platform, ingesting over 30,000 events per second.
*   Engineered Python ETL jobs, contributing to a 15% overall improvement in data processing speed.
*   Designed and built automated integration tests using pytest and GitHub Actions, enhancing code quality and deployment reliability.

---

**Projects**

**SmartTransit** | Python, Machine Learning
*   Designed and deployed a real-time bus arrival prediction model leveraging LSTM networks, achieving an 18% improvement in prediction accuracy.
*   Developed a robust data pipeline to clean, merge, and process over 2 million GPS data points from a public transit dataset.
*   Containerized the prediction system using Docker, demonstrating proficiency in scalable deployment strategies.

**PixelForge** | React, Node.js
*   Engineered a full-stack collaborative drawing platform featuring real-time canvas synchronization via WebSockets.
*   Implemented secure role-based access control and persistent project storage using MongoDB.
*   Optimized frontend performance, resulting in a 30% reduction in load times through strategic code splitting and caching techniques.

**GPU Weather Simulator** | C++, CUDA
*   Accelerated a fluid-dynamics-based weather simulation by 12x by developing optimized CUDA kernels.
*   Created custom, memory-efficient CUDA kernels for particle advection and temperature diffusion processes, demonstrating advanced performance optimization.

---

**Leadership & Activities**

**Computer Science Society — Project Team Lead**
*   Led a team of 6 students in developing a campus navigation application with advanced indoor routing capabilities.
*   Managed project lifecycle, including facilitating stand-up meetings, defining roadmaps, and conducting code reviews.

**Hackathons**
*   **Winner (1st Place), SD Hacks 2024:** Developed a wildfire-risk prediction dashboard utilizing satellite imagery analysis, showcasing rapid prototyping and data-driven problem-solving.

---

**Additional Information**

*   Strong interest in ML engineering, distributed systems, and large-scale backend systems, aligning with Tesla's data science and engineering focus. Eager to apply and expand knowledge in areas such as Transformers, LLMs, and data visualization tools like Tableau.
*   Fluent in English and conversational in Spanish.

---

**Decision Making Rationale:**

The strategy for revising this resume centered on precisely aligning the candidate's existing qualifications with the requirements of the Tesla Data Science Internship, as identified through job research. Key considerations included:

1.  **Direct Skill Mapping:** Skills directly possessed by the candidate (e.g., Python, SQL, PyTorch, TensorFlow, data pipelines, distributed systems, REST APIs) were prominently featured. For areas where the candidate had related experience or conceptual understanding but not direct tool usage (e.g., Airflow, Tableau, specific advanced ML models), phrases like "(Familiarity with concepts like Transformers, CNNs, GANs, sequence models)" and "(Conceptual understanding, eager to apply through tools like Tableau)" were incorporated. This addresses requirements without misrepresenting expertise.
2.  **Responsibility Alignment:** Bullet points in "Experience" and "Projects" were rephrased to echo Tesla's responsibilities. For example, "Implemented data processing modules for a telemetry monitoring platform ingesting 30K+ events/sec" directly relates to "Designing, developing, and maintaining data platforms." The "SmartTransit" project's data pipeline work aligns with "Building data pipelines for production dashboards."
3.  **Emphasis on Data Science & ML:** The candidate's strong software engineering foundation was contextualized to highlight Machine Learning and data-centric aspects. The "SmartTransit" project's prediction accuracy improvement and data processing showcase ML application and data wrangling. The "GPU Weather Simulator" demonstrates an understanding of computational efficiency vital for large-scale data processing.
4.  **Problem-Solving and Communication:** Tesla's emphasis on "strong problem-solving skills" and "excellent verbal and written communication" is implicitly demonstrated. Project descriptions highlight the process of defining problems and finding solutions ("refine problem statements," "determine data-driven solutions"). The leadership role, involving team management and roadmapping, also showcases these abilities. The winning hackathon project specifically mentions "data-driven problem-solving."
5.  **Work Ethic and Location:** The header was updated to include "Fremont, CA," acknowledging the on-site requirement. The "Additional Information" section was modified to convey a proactive and eager mindset ("Eager to apply and expand knowledge"), aligning with Tesla's desired "proactive and critical mindset, with flexibility and adaptability."
6.  **Quantifiable Impact:** All quantifiable achievements from the original resume were retained and highlighted to provide concrete evidence of the candidate's capabilities and impact.
7.  **Structure and Clarity:** The resume maintains a clear, hierarchical structure. Technical skills are categorized for easy review. Language is concise and action-oriented.
8.  **No Fabrication:** The core principle was to represent the candidate accurately. Areas of conceptual understanding or related experience were noted cautiously, avoiding any false claims of expertise.